In [1]:
# import tensorflow as tf
# tf.python.control_flow_ops = tf

from keras.models import Sequential, model_from_json, load_model
from keras.optimizers import *
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, Cropping2D, ELU,  MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Lambda
#from keras.layers.pooling import MaxPooling2D
#from keras.callbacks import EarlyStopping

from scipy.misc import imread, imsave
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
import numpy as np
import random
import csv
import cv2
print('Done')

Using TensorFlow backend.


Done


In [2]:
!pip install -q keras
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
print('Done')

E: Could not open lock file /var/lib/dpkg/lock - open (13: Permission denied)
E: Unable to lock the administration directory (/var/lib/dpkg/), are you root?
Done


In [4]:
from urllib.request import urlretrieve
import os
import numpy as np
from sklearn.preprocessing import LabelBinarizer
#from tqdm import tqdm
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

download('https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip','data.zip')
#download('https://github.com/UjjwalSaxena/Behavior-Cloning-DataSet-Ujjwal/raw/master/data1.zip','data1.zip')


print("All the files are downloaded")


def uncompress_features_labels(dir,name):
    if(os.path.isdir(name)):
        print('Data extracted:'+name)
    else:
        with ZipFile(dir) as zipf:
            zipf.extractall(name)
# uncompress_features_labels('data.zip','data')
#uncompress_features_labels('data1.zip','data1')

All the files are downloaded


In [54]:
!ls data1/data1






driving_log.csv  IMG


In [5]:
BATCH_SIZE = 64
NUMBER_OF_EPOCHS = 4
correction=0.1
PATH = ["data/data/","data1/data1/","data2/data2/","data3/data3/","data4/data4/","data5/data5/","data6/data6/"]
FileName=["data","data1","data2","data3","data4","data5","data6"]
CSV_FILE = "driving_log.csv"

DATA=[]
for index in range(0,len(PATH)):
    uncompress_features_labels(FileName[index]+'.zip',FileName[index])
    with open(PATH[index] + CSV_FILE) as csvfile:
        reader = csv.reader(csvfile)
        cntr = 0
        co=0
        for line in reader:
            co+=1
            if (cntr == 0):
                cntr += 1
                continue
            line[0]= PATH[index]+'IMG/'+line[0].split('/')[-1].split('\\')[-1]
            line[1]= PATH[index]+'IMG/'+line[1].split('/')[-1].split('\\')[-1]
            line[2]= PATH[index]+'IMG/'+line[2].split('/')[-1].split('\\')[-1]
            DATA.append(line)
        print(co)

TRAINING_DATA, VALIDATION_DATA = train_test_split(DATA, test_size = 0.15)


TOTAL_TRAIN_DATA = len(TRAINING_DATA)
TOTAL_VALID_DATA = len(VALIDATION_DATA)
print('Done')
print (TOTAL_TRAIN_DATA)
print(TOTAL_VALID_DATA)

Data extracted:data
8037
Data extracted:data1
1980
Data extracted:data2
4326
Data extracted:data3
829
Data extracted:data4
154
Data extracted:data5
559
Data extracted:data6
420
Done
13853
2445


In [6]:
def flipped(image, measurement):
    return np.fliplr(image), -measurement

def random_distort(img, angle):
    ''' 
    method for adding random distortion to dataset images, including random brightness adjust, and a random
    vertical shift of the horizon position
    '''
#     imgs=[]
#     msr=[]
    new_img = img.astype(float)
    # random brightness - the mask bit keeps values from going beyond (0,255)
    value = np.random.randint(-30, 30)
    if value > 0:
        mask = (new_img[:,:,0] + value) > 255 
    if value <= 0:
        mask = (new_img[:,:,0] + value) < 0
    new_img[:,:,0] += np.where(mask, 0, value)
    
#     imgs.append(new_img)
#     msr.append(angle)
    
    # random shadow - full height, random left/right side, random darkening
    
    h,w = new_img.shape[0:2]
    mid = np.random.randint(0,w)
    factor = np.random.uniform(0.6,0.8)
    if np.random.rand() > .5:
        new_img[:,0:mid,0] *= factor
    else:
        new_img[:,mid:w,0] *= factor
    
#     imgs.append(new_img)
#     msr.append(angle)
    
    # randomly shift horizon
    
    h,w,_ = new_img.shape
    horizon = 2*h/5
    v_shift = np.random.randint(-h/8,h/8)
    pts1 = np.float32([[0,horizon],[w,horizon],[0,h],[w,h]])
    pts2 = np.float32([[0,horizon+v_shift],[w,horizon+v_shift],[0,h],[w,h]])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    new_img = cv2.warpPerspective(new_img,M,(w,h), borderMode=cv2.BORDER_REPLICATE)
    
#     imgs.append(new_img)
#     msr.append(angle)
    
    return (new_img, angle)

In [7]:
def get_image(data):
  
    center_image = cv2.GaussianBlur(cv2.cvtColor(cv2.imread(data[0]), cv2.COLOR_BGR2RGB), (3,3), 0)
    left_image = cv2.GaussianBlur(cv2.cvtColor(cv2.imread(data[1]), cv2.COLOR_BGR2RGB), (3,3), 0)
    right_image = cv2.GaussianBlur(cv2.cvtColor(cv2.imread(data[2]), cv2.COLOR_BGR2RGB), (3,3), 0)

    images_arr=[center_image,left_image,right_image]
    correction_arr=[0, correction, -correction]
    # scale to 66x200x3 (same as nVidia)
    #new_img = cv2.resize(new_img,(200, 66), interpolation = cv2.INTER_AREA)
    images=[]
    measurements=[]
    for index in range(0,len(images_arr)):
        
        r= np.random.rand()
        steering_angle= float(data[3])+ correction_arr[index]
        
        images.append(images_arr[index])
        measurements.append(steering_angle)
#         if(r>0.5):
#             di, dm= random_distort(images_arr[index], steering_angle)
#             images.append(di)
#             measurements.append(dm)
        
#         if(steering_angle>=0.1):
        fi, fm= flipped(images_arr[index], steering_angle)
        images.append(fi)
        measurements.append(fm)
            
#         elif(r>0.33):
#             di, dm= random_distort(images_arr[index], steering_angle)
#             images.append(di)
#             measurements.append(dm)
#             if(r>0.66):
#                 di, dm= random_distort(fi, fm)
#                 images.append(di)
#                 measurements.append(dm)
        
    return images, measurements
print('Done')

Done


In [8]:
def generate_samples(data, batch_size):
    shuffle(data)
    while True:
        SIZE = len(data)
        for start in range(0, SIZE, batch_size):
            images, measurements = [], []
            for this_id in range(start, start + batch_size): 
                if this_id < SIZE:
                    image, measurement = get_image(data[this_id])
                    measurements+=measurement
                    images+=image
            #print(len(images))
            yield shuffle(np.array(images), np.array(measurements))



In [9]:
model = Sequential()

model.add(Cropping2D(cropping=((70,20),(0,0)), input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x/127.5)-1))
model.add(Convolution2D(24,5,5, border_mode='same', subsample=(2,2), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D(36,5,5, border_mode='same', subsample=(2,2), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D(48,5,5, border_mode='same', subsample=(2,2), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Convolution2D(64,3,3, border_mode='same', subsample=(1,1), activation="relu"))

model.add(Convolution2D(64,3,3, border_mode='same', subsample=(1,1),  activation="relu"))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))

model.add(Dense(50))

model.add(Dense(10))

model.add(Dense(1))

#opt = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.summary()
model.compile(loss='mse',optimizer=optimizers.Adam(lr=0.0001))



____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_1 (Cropping2D)        (None, 70, 320, 3)    0           cropping2d_input_1[0][0]         
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 70, 320, 3)    0           cropping2d_1[0][0]               
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 35, 160, 24)   1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 17, 80, 24)    0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
print('Training model...')
training_generator = generate_samples(TRAINING_DATA, batch_size = BATCH_SIZE)
validation_generator = generate_samples(VALIDATION_DATA, batch_size = BATCH_SIZE)
model.fit_generator(training_generator,
#                     TOTAL_TRAIN_DATA
                 samples_per_epoch = 28000,
                 validation_data = validation_generator,
                 nb_val_samples = TOTAL_VALID_DATA,
                 nb_epoch = NUMBER_OF_EPOCHS,
                 verbose = 1)

Training model...
Epoch 1/4
27648/28000 [============================>.] - ETA: 0s - loss: 0.0355

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


28032/28000 [==============================] - 40s - loss: 0.0355 - val_loss: 0.0361
Epoch 2/4
28032/28000 [==============================] - 38s - loss: 0.0316 - val_loss: 0.0224
Epoch 3/4
28206/28000 [==============================] - 38s - loss: 0.0304 - val_loss: 0.0331
Epoch 4/4
28032/28000 [==============================] - 38s - loss: 0.0297 - val_loss: 0.0310


In [11]:
print('Saving model...')
model.save("model19.h5")
print("Model Saved.")

Saving model...
Model Saved.
